In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Jupiter  MacOS
# BASE_DIR = "/Users/johnhanratty/ASLtest/asl-signs"  #"/Users/johnhanratty/ASLtest/asl-signs"
# WORKING_DIR = "/Users/johnhanratty/ASLtest"
# ARCHIVE_DIR = "/Users/johnhanratty/ASLtest"
# MODEL_DIR = "/Users/johnhanratty/ASLtest/models"

# !pip install nb_black --quiet
# %load_ext lab_black

# Colab
BASE_DIR = "/content/asl-signs"   #"/content/drive/MyDrive/GaggleSignLang/asl-signs"
WORKING_DIR = "/content/asl-work"
ARCHIVE_DIR = "/content/drive/MyDrive/GaggleSignLang"
MODEL_DIR = "/content/drive/MyDrive/GaggleSignLang/models"
# !pip install nb_black --quiet
# print('-----ok')
# %load_ext nb_black

# KAGGLE
# BASE_DIR = "/kaggle/input/asl-signs"
# WORKING_DIR = "/kaggle/working"
# ARCHIVE_DIR = "/kaggle/working"
# MODEL_DIR  = "/kaggle/working"
# !pip install nb_black --quiet --root-user-action=ignore
# %load_ext lab_black

import os
import gc
import shutil
import time

import json
from tqdm import tqdm
import numpy as np
import pandas as pd
import pickle
from random import seed, sample

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize

!pip install tsai --quiet
from tsai.all import *
from fastai.callback.tracker import EarlyStoppingCallback


import warnings
warnings.filterwarnings(action='ignore')

LANDMARK_FILES_DIR = f'{BASE_DIR}/train_landmark_files'
TRAIN_FILE = f"{BASE_DIR}/train.csv"

FRAMES_OUT = 32 # 16
PTS_IN_FRAME = 345
DIMC = [0,1,2]
DIMS = len(DIMC)
WORKERS = 0   # dataoader work var  0 for MAC, 4 for online


print('done')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.9/298.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.2 MB/s eta 0:00:00
done


In [4]:
import pickle

with open(f"{MODEL_DIR}/mod_trans32.pkl", 'rb') as f2:
        mod_trans = pickle.load(f2)

with open(f"{MODEL_DIR}/mod_incept32.pkl", 'rb') as f3:
       mod_incept = pickle.load(f3)

# PATH = f"{MODEL_DIR}/mod_ccn{FRAMES_OUT}.pt"
# mod_cnn = torch.load(PATH)


with open(f"{ARCHIVE_DIR}/truth_nan.pkl", 'rb') as handle:
    nan_truth = pickle.load(handle)


datax = np.load(f"{ARCHIVE_DIR}/cnn_data{FRAMES_OUT}.npy")
datay = np.load(f"{ARCHIVE_DIR}/cnn_labels.npy") 


Tdatax = np.load(f"{ARCHIVE_DIR}/tsai_data{FRAMES_OUT}.npy")
Tdatay = np.load(f"{ARCHIVE_DIR}/tsai_labels.npy") 
trainx, testx, trainy, testy, Trainx, Testx, Trainy, Testy, _, Truthy = train_test_split(datax, datay, Tdatax, Tdatay, nan_truth, test_size=0.15, random_state=42)





In [48]:
with open(f"{MODEL_DIR}/pred_tsai.pkl", 'rb') as f2:
        plist_tsai = pickle.load(f2)
with open(f"{MODEL_DIR}/pred_cnn.pkl", 'rb') as f2:
        plist_cnn = pickle.load(f2)


plist_cnn = plist_cnn.drop(["truth"], axis=1)

ens = pd.concat([plist_tsai, plist_cnn["cnn"]], axis=1)

ens['vote'] = np.where(ens['incept'] == ens['trans'], ens['incept'],
              np.where(ens['incept'] == ens['cnn'], ens['cnn'], 
              np.where(ens['cnn'] == ens["trans"], ens["trans"], ens['incept'])))
ens = ens.apply(pd.to_numeric)
print(ens.head())


   truth  incept  trans  cnn   vote
0  206.0   247.0  247.0   78  247.0
1   20.0    20.0   46.0   96   20.0
2  178.0    41.0  201.0  178   41.0
3  114.0   114.0  114.0  114  114.0
4  221.0   221.0  221.0  221  221.0


In [51]:

print("VOTE",
np.mean(np.where(ens["truth"] == ens["vote"], True, False)))

print("CNN",
np.mean(np.where(ens["truth"] == ens["cnn"], True, False)))
print("Incept",
np.mean(np.where(ens["truth"] == ens["incept"], True, False)))
print("Trans",
np.mean(np.where(ens["truth"] == ens["trans"], True, False)))





VOTE 0.7164832063223258
CNN 0.6898814563928873
Incept 0.7164832063223258
Trans 0.7348292407564211


In [38]:
type(ens["truth"])
print(ens.head())

   truth  incept  trans  cnn   vote    acc
0  206.0   247.0  247.0   78  247.0  False
1   20.0    20.0   46.0   96   46.0  False
2  178.0    41.0  201.0  178  201.0  False
3  114.0   114.0  114.0  114  114.0  False
4  221.0   221.0  221.0  221  221.0  False


In [6]:
y_hat_incept = mod_incept(torch.tensor(Testx).long())
y_hat_trans = mod_trans(torch.tensor(Testx).long())

RuntimeError: ignored

In [ ]:
# FIX TRUTH
preds['truth']=testy.astype(int)
preds.predi = preds.predi.astype(float).astype(int)
preds.predt = preds.predt.astype(float).astype(int)

sm = nn.Softmax(dim=1) 
prob_cnns = sm(torch.Tensor(prob_cnn)).numpy()
preds['predc'] = np.argmax(prob_cnns, axis=1)

preds["probi"] = prob_in.numpy().max(axis=1)
preds["probt"] = prob_tran.numpy().max(axis=1)
preds["probc"] = prob_cnns.max(axis=1)


In [ ]:
preds.head()


,truth,nans,idx,predt,predi,predc,probi,probt,probc
80305,206,20,80305,247,206,78,0.627538,0.711359,0.384106
80306,20,42,80306,20,96,96,0.426852,0.784956,0.486097
80307,178,16,80307,26,178,200,0.641837,0.267397,0.072054
80308,114,50,80308,114,114,114,0.888171,0.965296,0.977423
80309,221,64,80309,221,221,221,0.954264,0.982313,0.877944


In [ ]:
print("SINGLE MODEL ACCURACY")
print("Tran", np.mean(preds['truth'] == preds['predt']), 
      "In", np.mean(preds['truth'] == preds['predi']),
      "CNN",  np.mean(preds['truth'] == preds['predc']))


SINGLE MODEL ACCURACY
Tran 0.735182049110923 In 0.7398391193903472 CNN 0.6808495625176404


In [ ]:
# Choose highest prob prediction from ALL models
def highest_prob(row):
  if (row['probi'] > row['probt']) & (row['probi'] > row['probc']):
    return row['predi']
  elif row['probt'] > row['probc']:
    return row['predt']
  else:
    return row['predc']

print("ALL MODELS")
preds["hpall"] = preds.apply(lambda row: highest_prob(row), axis=1)

print(np.mean(preds.truth==preds.hpall))

ALL MODELS
0.749435506632797


In [ ]:
# Choose highest prob prediction from PROBI & PROBT
def highest_prob(row):
  if (row['probi'] > row['probt']):
    return row['predi']
  else:
    return row['predt']

preds["hpit"] = preds.apply(lambda row: highest_prob(row), axis=1)
print("I & T Models")
print(np.mean(preds.truth==preds.hpit))

I & T Models
0.7452018063787751


In [ ]:
# Choose highest prob prediction from PROBI & PROBC
def highest_prob(row):
  if (row['probi'] > row['probc']):
    return row['predi']
  else:
    return row['predc']

preds["hpic"] = preds.apply(lambda row: highest_prob(row), axis=1)
print("i & c Models")
print(np.mean(preds.truth==preds.hpic))

i & c Models
0.7444961896697714


In [ ]:
# Choose highest prob prediction from PROBT & PROBC
def highest_prob(row):
  if (row['probt'] > row['probc']):
    return row['predt']
  else:
    return row['predc']

preds["hptc"] = preds.apply(lambda row: highest_prob(row), axis=1)

print(np.mean(preds.truth==preds.hptc))

0.7353231724527237


In [ ]:
preds['incep'] = preds.truth == preds.predi
preds['trans'] = preds.truth == preds.predt
preds['cnnag'] = preds.truth == preds.predc

In [ ]:
preds.head()

,truth,nans,idx,predt,predi,predc,probi,probt,probc,hpall,hpit,hpic,hptc,incep,trans,cnnag
80305,206,20,80305,247,206,247,0.627538,0.711359,0.735845,247.0,247.0,247.0,247.0,True,False,False
80306,20,42,80306,20,96,96,0.426852,0.784956,0.357823,20.0,20.0,96.0,20.0,False,True,False
80307,178,16,80307,26,178,151,0.641837,0.267397,0.042810,178.0,178.0,178.0,26.0,True,False,False
80308,114,50,80308,114,114,114,0.888171,0.965296,0.973645,114.0,114.0,114.0,114.0,True,True,True
80309,221,64,80309,221,221,221,0.954264,0.982313,0.869086,221.0,221.0,221.0,221.0,True,True,True


In [ ]:
fcnt = preds[preds.nans < 6].groupby('nans').agg(iacc = ("incep","mean"),
                                                 cnt = ("idx","count"))

fcnt

,iacc,cnt
nans,,
0,0.833333,6
1,0.750000,8
2,0.863636,22
3,0.666667,21
4,0.689655,29
5,0.775000,40


In [ ]:
preds[preds.nans < 4].groupby('truth').agg(iacc = ("incep","mean"),
                                cnt = ("idx","count"))



,iacc,cnt
truth,,
9,1.000000,1
19,1.000000,1
22,1.000000,1
37,1.000000,1
38,1.000000,1
44,1.000000,1
48,1.000000,1
51,1.000000,1
62,1.000000,2


In [ ]:
# ENSEMBLE VOTING
print("2 out of 3",)